## Feature engineering
1. last encounter date
2. last bmi
3. insurance 

In [ ]:
## load libraries 
# Import standard Python libraries
import getpass  
import re 
import json 
import sys  

# Import data analysis and visualization libraries
import pandas as pd 
import numpy as np  
import seaborn as sns  
import matplotlib.pyplot as plt  

# Import datetime utilities
from datetime import datetime, timedelta  

# Import libraries for Google BigQuery
import pandas_gbq as pgbq  
from google.cloud import bigquery  

# Import operating system utilities
import os  

# Import SQLAlchemy for database connection
from sqlalchemy import create_engine  



In [ ]:
# Configure BigQuery project and environment
project_id = ''  # BigQuery project ID
client = bigquery.Client(project=project_id)  # Initialize BigQuery client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''  # Authentication credentials
os.environ['GCLOUD_PROJECT'] = project_id  # Set the project environment

# Define database and datasets
db = ""  # Main database
stanford_ds = ""  # Stanford dataset
yh_ds = ""  # Custom dataset

In [ ]:
## custom functions 
# Save a query result to a BigQuery table
def save_table(project_id, yh_ds, new_table_name, query):
    table_id = f"{project_id}.{yh_ds}.{new_table_name}"  # Full table path
    job_config = bigquery.QueryJobConfig(destination=table_id)
    job_config.write_disposition = "WRITE_TRUNCATE"  # Overwrite the table if it exists
    query_job = client.query(query, job_config=job_config)  # Run the query
    query_job.result()  # Wait for the query job to complete
    print(f"Query results loaded to the table {table_id}")

# Load a table from BigQuery into a Pandas DataFrame
def load_pgbq(project_id, yh_ds, table_name):
    sql_query = f"SELECT * FROM {project_id}.{yh_ds}.{table_name}"  # Query to fetch all rows
    return_df = pgbq.read_gbq(sql_query, dialect="standard")  # Load table as DataFrame
    print(f"{project_id}.{yh_ds}.{table_name} is loaded")
    return return_df

# Upload a Pandas DataFrame to BigQuery
def upload_pgbq(project_id, yh_ds, table_name, df):
    table_id = f"{yh_ds}.{table_name}"  # Target table path
    pgbq.to_gbq(df, table_id, project_id=project_id, if_exists='replace')  # Upload DataFrame
    print(f"DataFrame is uploaded as {project_id}.{yh_ds}.{table_name}")

# Remove a BigQuery table
def remove_table(project_id, yh_ds, table_name):
    table_id = f"{project_id}.{yh_ds}.{table_name}"  # Full table path
    client.delete_table(table_id, not_found_ok=True)  # Delete the table if it exists
    print(f"Deleted table '{table_id}'.")

## last encounter and bmi information from the encounter table 

In [ ]:
## last encounter date and number of encounters 
# Define dataset and table names
stanford_ds = "stanfordmed_datalake"
yh_ds = "YH_dementia"
yh_encounter = "encounter_07312024"
yh_cohort = "dementia_pat_exposure_group_07312024"
yh_new_cohort = "dementia_pat_last_encounter_07312024"
cut_off_date = '2024-07-31'
# SQL query to create or replace the new table with last encounter date and BMI
sql_query = f"""
CREATE OR REPLACE TABLE {project_id}.{yh_ds}.{yh_new_cohort} AS

SELECT 
    pat.*,  -- Include all columns from the patient cohort
    vc.visits_before,  -- Number of visits before the diagnosis date
    vc.visits_after,  -- Number of visits after the diagnosis date
    vc.last_encounter_date,  -- last encounter date before the cut off date 
    bmi_data.last_bmi_before_diagnosis  -- Most recent BMI before diagnosis
FROM `{project_id}.{yh_ds}.{yh_cohort}` AS pat  -- Patient cohort as the main table

LEFT JOIN (
    -- Subquery to calculate visits before/after diagnosis and last encounter date
    SELECT 
        enc.pat_deid,  -- Patient ID
        SUM(CASE WHEN TIMESTAMP(enc.contact_date) < TIMESTAMP(cohort.diagnosis_date) THEN 1 ELSE 0 END) AS visits_before,  -- Count visits before diagnosis
        SUM(CASE WHEN TIMESTAMP(enc.contact_date) >= TIMESTAMP(cohort.diagnosis_date) THEN 1 ELSE 0 END) AS visits_after,   -- Count visits after diagnosis
        MAX(CASE WHEN TIMESTAMP(enc.contact_date) <= TIMESTAMP('{cut_off_date}') THEN enc.contact_date END) AS last_encounter_date  -- Last encounter date before the cutoff
    FROM `{project_id}.{yh_ds}.{yh_encounter}` AS enc  -- Encounter data
    JOIN `{project_id}.{yh_ds}.{yh_cohort}` AS cohort  -- Join with patient cohort to access diagnosis date
        ON enc.pat_deid = cohort.pat_deid
    GROUP BY enc.pat_deid, cohort.diagnosis_date  -- Group by patient ID and diagnosis date
) AS vc ON pat.pat_deid = vc.pat_deid  -- Join the aggregated visit data to the patient table

LEFT JOIN (
    -- Subquery to get the last BMI value before diagnosis
    SELECT 
        ranked.pat_deid,  -- Patient ID
        ranked.bmi AS last_bmi_before_diagnosis  -- Last BMI value before diagnosis
    FROM (
        -- Subquery to rank BMI records by contact date
        SELECT 
            enc_sub.pat_deid,  -- Patient ID
            enc_sub.bmi,  -- BMI value
            enc_sub.contact_date,  -- Contact date for the BMI record
            ROW_NUMBER() OVER (PARTITION BY enc_sub.pat_deid ORDER BY TIMESTAMP(enc_sub.contact_date) DESC) AS rnk  -- Rank rows by most recent contact date
        FROM `{project_id}.{yh_ds}.{yh_encounter}` AS enc_sub  -- Encounter data
        WHERE enc_sub.bmi IS NOT NULL  -- Include only rows with BMI values
        AND TIMESTAMP(enc_sub.contact_date) < (
            SELECT MIN(cohort_sub.diagnosis_date)  -- Get the earliest diagnosis date for the patient
            FROM `{project_id}.{yh_ds}.{yh_cohort}` AS cohort_sub
            WHERE cohort_sub.pat_deid = enc_sub.pat_deid
        )
    ) AS ranked
    WHERE ranked.rnk = 1  -- Select only the most recent BMI record
) AS bmi_data ON pat.pat_deid = bmi_data.pat_deid;  -- Join the last BMI data to the patient table
"""

# Execute the query
query_job = client.query(sql_query)
query_job.result()


## insurance information 

In [ ]:
shc_coverage = 'shc_coverage'
yh_cohort = 'dementia_pat_last_encounter_07312024'
shc_encounter = 'shc_encounter'
sql_query = f"""
WITH ranked_coverages AS (
    SELECT 
        cov.pat_deid,
        cov.fin_class_name,
        ROW_NUMBER() OVER (PARTITION BY cov.pat_deid ORDER BY DATETIME(cov.mem_eff_from_date) DESC) AS rn
    FROM `{project_id}.{stanford_ds}.{shc_coverage}` AS cov
    JOIN `{project_id}.{yh_ds}.{yh_cohort}` AS pat
      ON cov.pat_deid = pat.pat_deid
    -- Filter where the exposure date falls within the coverage period
    WHERE DATETIME(pat.post_onset_exposure_start_time) >= DATETIME(cov.mem_eff_from_date)
)
SELECT 
    pat.*,
    enc.last_bmi_before_exposure,
    -- Map insurance type to major categories, handle null values from non-matching rows
    CASE 
        WHEN r_cov.fin_class_name LIKE '%WORKER%' THEN 'Private' -- private 
        WHEN r_cov.fin_class_name IN ('BLUE SHIELD', 'MANAGED CARE', 'BLUE CROSS') THEN 'Private' -- private
        WHEN r_cov.fin_class_name IN ('MEDICARE', 'MEDICARE MANAGED CARE') THEN 'Medicare_Medicaid' -- medicare 
        WHEN r_cov.fin_class_name IN ('MEDI-CAL', 'MEDI-CAL MANAGED CARE', 'MEDICAID') THEN 'Medicare_Medicaid' -- medicaid 
        WHEN r_cov.fin_class_name IN ('OTHER', '*UNSPECIFIED FINANCIAL CLASS') THEN 'Other' -- other 
        ELSE 'Missing' -- unknown, Handles any unmapped values
    END AS mapped_insurance_type
FROM `{project_id}.{yh_ds}.{yh_cohort}` AS pat
LEFT JOIN (
    SELECT 
        enc.pat_deid,
        ANY_VALUE(enc.bmi) AS last_bmi_before_exposure
    FROM `{project_id}.{stanford_ds}.{shc_encounter}` AS enc
    JOIN `{project_id}.{yh_ds}.{yh_cohort}` AS pat_sub
      ON enc.pat_deid = pat_sub.pat_deid
    WHERE DATETIME(enc.contact_date) < DATETIME(pat_sub.post_onset_exposure_start_time)
      AND enc.bmi IS NOT NULL
    GROUP BY enc.pat_deid
) enc ON pat.pat_deid = enc.pat_deid
-- Join the ranked coverages table where rn = 1 (most recent coverage)
LEFT JOIN ranked_coverages r_cov
ON pat.pat_deid = r_cov.pat_deid AND r_cov.rn = 1;
"""

In [ ]:
save_table(project_id, yh_ds, "dementia_pat_coverage_07312024", sql_query)